In [1]:
import os

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [2]:
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum



class ImageDescription(BaseModel):
    description: str = Field(description="The description of the image")
    confidence: float = Field(description="The confidence score of the description")












In [4]:
import base64

from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

# Example using a public URL (remains the same)
message_url = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "Describe the image at the URL.",
        },
        {"type": "image_url", "image_url": "https://picsum.photos/seed/picsum/200/300"},
    ]
)
result_url = llm.invoke([message_url])#.with_structured_output(ImageDescription)
print(f"Response for URL image: {result_url.content}")

# # Example using a local image file encoded in base64
# image_file_path = "/Users/philschmid/projects/google-gemini/langchain/docs/static/img/agents_vs_chains.png"

# with open(image_file_path, "rb") as image_file:
#     encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

# message_local = HumanMessage(
#     content=[
#         {"type": "text", "text": "Describe the local image."},
#         {"type": "image_url", "image_url": f"data:image/png;base64,{encoded_image}"},
#     ]
# )
# result_local = llm.invoke([message_local])
# print(f"Response for local image: {result_local.content}")

Response for URL image: Here's a description of the image:

The image is a landscape photograph featuring a snow-covered mountain and a sky filled with soft, pastel-colored clouds. 

*   **Foreground:** The bottom portion of the image is dominated by a large, snow-covered slope, appearing dark blue due to the shadows.
*   **Midground:** A rocky, snow-covered mountain peak is visible on the right side of the image. The snow on the peak is illuminated by the sunlight.
*   **Background:** The sky is filled with a beautiful array of clouds, colored in soft pinks, oranges, and purples, suggesting either sunrise or sunset. The overall tone of the image is serene and peaceful.


In [3]:
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum

class LeafType(str, Enum):
    TOMATO = "tomato"
    NOT_TOMATO = "not_tomato"
    UNCLEAR = "unclear"

class HealthStatus(str, Enum):
    HEALTHY = "healthy"
    DISEASED = "diseased"
    STRESSED = "stressed"
    UNCLEAR = "unclear"
    NOT_APPLICABLE = "not_applicable"  # For non-tomato leaves

class DiseaseType(str, Enum):
    EARLY_BLIGHT = "early_blight"
    LATE_BLIGHT = "late_blight"
    BACTERIAL_SPOT = "bacterial_spot"
    SEPTORIA_LEAF_SPOT = "septoria_leaf_spot"
    YELLOW_LEAF_CURL = "yellow_leaf_curl"
    MOSAIC_VIRUS = "mosaic_virus"
    NUTRIENT_DEFICIENCY = "nutrient_deficiency"
    PEST_DAMAGE = "pest_damage"
    OTHER = "other"

class TomatoLeafAnalysis(BaseModel):
    """Structured output for tomato leaf health analysis"""
    
    # Primary classification
    is_tomato_leaf: LeafType = Field(
        description="Whether the image shows a tomato leaf or not"
    )
    
    confidence_score: float = Field(
        ge=0.0, le=1.0,
        description="Confidence level in the tomato leaf identification (0-1)"
    )
    
    # Health assessment (only if tomato leaf)
    health_status: Optional[HealthStatus] = Field(
        default=None,
        description="Overall health status of the tomato leaf"
    )
    
    # Disease identification
    diseases_detected: List[DiseaseType] = Field(
        default_factory=list,
        description="List of diseases or issues detected"
    )
    
    # Detailed analysis
    symptoms_observed: List[str] = Field(
        default_factory=list,
        description="Specific symptoms visible on the leaf"
    )
    
    severity_level: Optional[str] = Field(
        default=None,
        description="Severity level: mild, moderate, severe"
    )
    
    # Recommendations
    treatment_recommendations: List[str] = Field(
        default_factory=list,
        description="Specific treatment recommendations"
    )
    
    prevention_tips: List[str] = Field(
        default_factory=list,
        description="Prevention measures for future care"
    )
    
    # Additional insights
    additional_notes: Optional[str] = Field(
        default=None,
        description="Any additional observations or context"
    )

# Usage example
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Create message for tomato leaf analysis
message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": """Analyze this image for tomato leaf health. Please provide:
            1. Confirm if this is a tomato leaf (if not, set health_status to 'not_applicable')
            2. If it IS a tomato leaf, assess the health status
            3. If it IS a tomato leaf, identify any diseases or problems
            4. If it IS a tomato leaf, list specific symptoms you observe
            5. If it IS a tomato leaf, provide treatment recommendations
            6. If it IS a tomato leaf, suggest prevention measures
            
            IMPORTANT: If this is NOT a tomato leaf, set health_status to 'not_applicable' and leave treatment/prevention recommendations empty.""",
        },
        {"type": "image_url", "image_url": "https://picsum.photos/seed/picsum/200/300"},  # Using the working URL
    ]
)

# Get structured output - call with_structured_output on the model, not the result
structured_llm = llm.with_structured_output(TomatoLeafAnalysis)
result = structured_llm.invoke([message])

# Access the structured data
print(f"Is tomato leaf: {result.is_tomato_leaf}")
print(f"Confidence: {result.confidence_score}")

if result.is_tomato_leaf == LeafType.TOMATO:
    print(f"Health status: {result.health_status}")
    print(f"Diseases detected: {result.diseases_detected}")
    print(f"Symptoms: {result.symptoms_observed}")
    print(f"Treatment recommendations: {result.treatment_recommendations}")
    print(f"Prevention tips: {result.prevention_tips}")
else:
    print("This is not a tomato leaf - no health analysis performed")
    print(f"Additional notes: {result.additional_notes}")

Is tomato leaf: not_tomato
Confidence: 0.0
This is not a tomato leaf - no health analysis performed
Additional notes: None
